In [1]:
!pip install findspark
!pip install pyspark
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=e1c5f33254897183628594608fd8f213abefb596d97c697c681efe482c9de965
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=712eb286a0f0b35aa62d981846c15bacb3b2e29f027cda1174ce8db33b7ea1a9
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [162]:
import pyspark
import findspark
import spark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder.getOrCreate()

In [ ]:
lista = [n for n in range(1,101)]

for numero in lista:
    div3 = numero-((numero//3)*3)
    div5 = numero-((numero//5)*5)

    if div3 == 0 and div5 == 0:
        print('BI')
    elif div3 == 0:
        print('B')
    elif div5 == 0:
        print('I')
    else:
        print(numero)

# SERIE II

In [ ]:
cards = spark.read.option('inferSchema','true').option('header','true').csv('Credit Card Customer Data.csv')

In [ ]:
lower_column_name = [col.lower().replace(' ', '_') for col in cards.columns]
cards = cards.toDF(*lower_column_name)
cards.columns

['sl_no',
 'customer_key',
 'avg_credit_limit',
 'total_credit_cards',
 'total_visits_bank',
 'total_visits_online',
 'total_calls_made']

### 1. identificar al cliente con mayor limite de credito

In [ ]:
cards.select(
    'customer_key',
    'avg_credit_limit'
)\
.groupBy('customer_key').agg(max('avg_credit_limit').alias('max_credit_limit'))\
.orderBy(desc('max_credit_limit'))\
.limit(1)\
.show()

+------------+----------------+
|customer_key|max_credit_limit|
+------------+----------------+
|       33837|          200000|
+------------+----------------+



### 2. Basandose en el numero de visitas al banco, visitas online y total de llamadas. Crear un nuevo campo que pueda clasificar a los clientes de la siguiente manera:

  ##### a. Si el total de visitas al banco es mayor que otros = tradicional
  ##### b. Si el total de visitas online es mayor que otros = digital
  ##### c. Si el total de llamadas es mayor que otros = telefonico
  ##### d. Si tiene el mismo valor en dos o tres canales = Multicanal



In [ ]:
cards.select(
    'customer_key',
    'total_visits_bank',
    'total_visits_online',
    'total_calls_made',
    when((col('total_visits_bank') > col('total_visits_online')) & (col('total_visits_bank') > col('total_calls_made')), 'Tradicional').\
    when((col('total_visits_online') > col('total_visits_bank')) & (col('total_visits_online') > col('total_calls_made')), 'Digital').\
    when((col('total_calls_made') > col('total_visits_bank')) & (col('total_calls_made') > col('total_visits_online')), 'Telefonico').\
    otherwise('Multicanal').\
    alias('customer_type')
).show()

+------------+-----------------+-------------------+----------------+-------------+
|customer_key|total_visits_bank|total_visits_online|total_calls_made|customer_type|
+------------+-----------------+-------------------+----------------+-------------+
|       87073|                1|                  1|               0|   Multicanal|
|       38414|                0|                 10|               9|      Digital|
|       17341|                1|                  3|               4|   Telefonico|
|       40496|                1|                  1|               4|   Telefonico|
|       47437|                0|                 12|               3|      Digital|
|       58634|                0|                  1|               8|   Telefonico|
|       48370|                0|                 11|               2|      Digital|
|       37376|                0|                  1|               1|   Multicanal|
|       82490|                0|                  2|               2|   Mult

### 3. ¿Para qué podría utilizar una empresa del sector bancario el campo creado en el punto 2?

In [ ]:
'''
Para poder saber el medio favorito de interacción del cliente con el banco y por este, poder ofrecer nuevos productos o servicios.
Así como crear estrategias de mercado dependiendo la via favorita de los clientes en general.
'''

### 4. Sumar los campos total_visits_banck, total_visits_online, total_calls_mode para identificar a los 3 clientes con el mayor total.

In [ ]:
cards.select(
    'customer_key',
    (col('total_visits_bank') + col('total_visits_online') + col('total_calls_made')).alias('total_customer')
)\
.orderBy(desc('total_customer'))\
.limit(3)\
.show()

+------------+--------------+
|customer_key|total_customer|
+------------+--------------+
|       38414|            19|
|       99843|            18|
|       90839|            18|
+------------+--------------+



### 5. ¿Cuántas tarjetas de credito hay en total? ¿Cuántos clientes tiene la empresa? ¿Cuáñ es el límite total?

In [ ]:
cards.select(
    count('sl_no').alias('total_tarjetas'),
    count_distinct('customer_key').alias('conteo_clientes'),
    sum('avg_credit_limit').alias('total_limite')
).show()

+--------------+---------------+------------+
|total_tarjetas|conteo_clientes|total_limite|
+--------------+---------------+------------+
|           660|            655|    22819000|
+--------------+---------------+------------+



### 6. Crear una nueva estructura dataframe llamada: df_cc_filter con todos los clientes que tengan menos de 3 tarjetas de credito.

In [ ]:
df_cc_filter = cards.groupBy('customer_key').agg(count('customer_key').alias('customer_cards')).filter(col('customer_cards') < 3)
df_cc_filter.show()

+------------+--------------+
|customer_key|customer_cards|
+------------+--------------+
|       41946|             1|
|       48510|             1|
|       49331|             1|
|       64519|             1|
|       45341|             1|
|       39285|             1|
|       82164|             1|
|       34103|             1|
|       29058|             1|
|       18595|             1|
|       26767|             1|
|       44398|             1|
|       47496|             1|
|       97109|             1|
|       14377|             1|
|       34495|             1|
|       87219|             1|
|       90586|             1|
|       25742|             1|
|       17036|             1|
+------------+--------------+
only showing top 20 rows



# SERIE III

### 1. Leer los archivos en su respectivo DF de pyspark y mostrar el esquema de cada DF

In [3]:
product = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Product.csv')
region = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Region.csv')
reseller = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Reseller.csv')
sales = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Sales.csv')
salesperson = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Salesperson.csv')
salespersonRegion = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('SalespersonRegion.csv')
targets = spark.read.option('inferSchema','true').option('header','true').option('delimiter','\t').csv('Targets.csv')

In [ ]:
product.printSchema()

root
 |-- productkey: integer (nullable = true)
 |-- product: string (nullable = true)
 |-- standard_cost: float (nullable = true)
 |-- color: string (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- category: string (nullable = true)
 |-- background_color_format: string (nullable = true)
 |-- font_color_format: string (nullable = true)



In [ ]:
region.printSchema()

root
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Group: string (nullable = true)



In [ ]:
reseller.printSchema()

root
 |-- ResellerKey: integer (nullable = true)
 |-- Business Type: string (nullable = true)
 |-- Reseller: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State-Province: string (nullable = true)
 |-- Country-Region: string (nullable = true)



In [ ]:
sales.printSchema()

root
 |-- SalesOrderNumber: string (nullable = true)
 |-- OrderDate: date (nullable = true)
 |-- ProductKey: integer (nullable = true)
 |-- ResellerKey: integer (nullable = true)
 |-- EmployeeKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Unit_Price: float (nullable = true)
 |-- Sales: float (nullable = true)
 |-- Cost: float (nullable = true)



In [ ]:
salesperson.printSchema()

root
 |-- EmployeeKey: integer (nullable = true)
 |-- EmployeeID: integer (nullable = true)
 |-- Salesperson: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- UPN: string (nullable = true)



In [ ]:
salespersonRegion.printSchema()

root
 |-- EmployeeKey: integer (nullable = true)
 |-- SalesTerritoryKey: integer (nullable = true)



In [ ]:
targets.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- Target: string (nullable = true)
 |-- TargetMonth: date (nullable = true)



### 2. Realizar el proceso de limpieza de datos en caso de ser necesario.

In [4]:
def format_column_names(columns):
  return [col.replace(' ', '_').replace('-','_') for col in columns]

In [5]:
# Aplicando formato a columnas
product = product.toDF(*format_column_names(product.columns))

# Cast correcto a columnas
product = product.select(
    col('productkey').cast('int').alias('productkey'),
    'product',
    regexp_extract('standard_cost', r'([0-9]+).([0-9]+)', 0).cast('float').alias('standard_cost'),
    'color',
    'subcategory',
    'category',
    'background_color_format',
    'font_color_format'
)

In [6]:
region = region.select(
    col('SalesTerritoryKey').cast('int'),
    'Region',
    'Country',
    'Group'
)

region = region.toDF(*format_column_names(region.columns))

In [7]:
reseller = reseller.toDF(*format_column_names(reseller.columns))

In [8]:
# Función que convierte fecha en letras a numeros en string para poder aplicar un cast tipo date

@udf(returnType=StringType())
def date_to_str_fun(date):
    months = ['january','february','march','april','may','june','july','august','september','october','november','december']

    a = date.split(', ')
    b = [x.split(' ') for x in a[1:]]
    year_var = a[2]
    month_var = months.index(b[0][0].lower())+1
    day_var = b[0][1]

    if month_var < 10: month_var = f'0{str(month_var)}'
    if int(day_var) < 10: day_var = f'0{day_var}'

    return f'{str(year_var)}-{str(month_var)}-{str(day_var)}'

In [170]:
sales = sales.toDF(*format_column_names(sales.columns))

# Nuevo DF sales con formato correcto a los datos
sales = sales.select(
    'SalesOrderNumber',
    date_to_str_fun('OrderDate').alias('OrderDate'),
    'ProductKey',
    'ResellerKey',
    'EmployeeKey',
    'SalesTerritoryKey',
    'Quantity',
    regexp_extract(regexp_replace('Unit_Price', ',', ''), r'([0-9]+).([0-9]+)', 0).cast(DecimalType(38,2)).alias('Unit_Price'),
    regexp_extract(regexp_replace('Sales', ',', ''), r'([0-9]+).([0-9]+)', 0).cast(DecimalType(38,2)).alias('Sales'),
    regexp_extract(regexp_replace('Cost', ',', ''), r'([0-9]+).([0-9]+)', 0).cast(DecimalType(38,2)).alias('Cost')
)

In [10]:
targets = targets.select(
    'EmployeeID',
    regexp_extract(regexp_replace('Target',',',''), r'([0-9]+).([0-9]+)',0).alias('Target'),
    date_to_str_fun('TargetMonth').alias('TargetMonth')
)

### 3. mostrar las ventas por región comparado anualmente nombre_region, año 2017, 2018, 2019, 2020

In [11]:
sales_region = sales.join(region, 'SalesTerritoryKey')

In [171]:
(sales_region\
.select(
    col('region').alias('nombre_region'),
    year('OrderDate').alias(f'year'),
    'Sales'
)\
.groupBy('nombre_region')\
.pivot('year')\
.sum('Sales')\
.orderBy('nombre_region'))\
.select(
    'nombre_region',
    round(coalesce('2017', lit(0)),2).alias('año_2017'),
    round(coalesce('2018', lit(0)),2).alias('año_2018'),
    round(coalesce('2019', lit(0)),2).alias('año_2019'),
    round(coalesce('2020', lit(0)),2).alias('año_2020')
).show()

+--------------+----------+----------+----------+----------+
| nombre_region|  año_2017|  año_2018|  año_2019|  año_2020|
+--------------+----------+----------+----------+----------+
|     Australia|       0.0|       0.0| 875621.81| 515331.02|
|        Canada|1514228.89|4860350.59|5689304.06|1808034.36|
|       Central| 951799.96|2635029.32|3015528.53|1029523.03|
|        France|       0.0| 860559.52|2405726.05| 1260801.1|
|       Germany|       0.0|       0.0|1127531.52| 750148.85|
|     Northeast| 568552.28|2453082.15|2875491.58| 816863.35|
|     Northwest|1690237.49|3502111.02|4686781.48|2124429.43|
|     Southeast|1450813.31|2837964.04|2443724.73| 904561.46|
|     Southwest|1894009.67| 6329568.6|7187096.67| 2587513.1|
|United Kingdom|       0.0| 843795.94|2186342.98| 852025.01|
+--------------+----------+----------+----------+----------+



### 4. Identificar los dos productos con mas cantidades vendidas en cada año.

In [39]:
group_product_sales = sales.join(product, 'ProductKey')\
                      .groupBy(year('OrderDate').alias('year'), 'ProductKey', 'product').agg(sum('Quantity').alias('product_sales'))

In [41]:
from pyspark.sql.window import Window

product_espec = Window.partitionBy('year').orderBy(desc('product_sales'))
group_product_sales.withColumn('rn', row_number().over(product_espec)).filter(col('rn') <= 2).show(truncate=False)

+----+----------+-------------------------------+-------------+---+
|year|ProductKey|product                        |product_sales|rn |
+----+----------+-------------------------------+-------------+---+
|2017|218       |Mountain Bike Socks, M         |563          |1  |
|2017|223       |AWC Logo Cap                   |520          |2  |
|2018|470       |Full-Finger Gloves, L          |1942         |1  |
|2018|233       |Long-Sleeve Logo Jersey, L     |1432         |2  |
|2019|471       |Classic Vest, S                |2347         |1  |
|2019|491       |Short-Sleeve Classic Jersey, XL|2052         |2  |
|2020|471       |Classic Vest, S                |1349         |1  |
|2020|491       |Short-Sleeve Classic Jersey, XL|1107         |2  |
+----+----------+-------------------------------+-------------+---+



### 5. Encontrar la diferencia entre la venta y el costo de cada orden e identificar por medio de un flag las ordenes que generaron una perdida para la empresa.



In [172]:
perdidas = (sales.select(
    'SalesOrderNumber',
    (col('Sales') - col('Cost')).alias('diff')
)\
.groupBy('SalesOrderNumber')\
.agg(sum('diff').alias('total_diff')))\
.select(
    'SalesOrderNumber',
    round('total_diff',2).alias('total_diff'),
    when(col('total_diff') < 0, 1).otherwise(0).alias('diff_flag')
)\
.filter(col('diff_flag') == 1)

perdidas.show()

+----------------+----------+---------+
|SalesOrderNumber|total_diff|diff_flag|
+----------------+----------+---------+
|         SO44521|    -46.00|        1|
|         SO57042|  -2276.00|        1|
|         SO63277|   -378.00|        1|
|         SO45291|   -289.00|        1|
|         SO46086|   -141.00|        1|
|         SO57159|    -99.00|        1|
|         SO46940|     -4.00|        1|
|         SO47721|     -4.00|        1|
|         SO69506|   -154.00|        1|
|         SO49123|   -928.00|        1|
|         SO69467|   -232.00|        1|
|         SO63195|   -616.00|        1|
|         SO65308|    -89.00|        1|
|         SO57175|    -62.00|        1|
|         SO48764|   -269.00|        1|
|         SO51789| -10273.00|        1|
|         SO51804|    -26.00|        1|
|         SO49859|   -100.00|        1|
|         SO44101|    -78.00|        1|
|         SO67311|   -342.00|        1|
+----------------+----------+---------+
only showing top 20 rows



### 6. ¿Cual es la categoria con mas ganancias reportadas?

In [72]:
sales_product = sales.join(product, 'ProductKey')

In [173]:
(sales_product.select(
    'category',
    (col('Sales') - col('Cost')).alias('diff')
)\
.groupBy('category')\
.agg(sum('diff').alias('total_diff')))\
.select(
    'Category',
    round('total_diff',2).alias('total_revenue'),
)\
.orderBy(desc('total_diff'))\
.limit(1)\
.show()

+----------+-------------+
|  Category|total_revenue|
+----------+-------------+
|Components|   1001235.46|
+----------+-------------+



### 7. Mostrar los numeros de orden de ventas que generaron un perdida para la empresa.

In [83]:
perdidas.select('SalesOrderNumber').show()

+----------------+
|SalesOrderNumber|
+----------------+
|         SO44521|
|         SO57042|
|         SO63277|
|         SO45291|
|         SO46086|
|         SO57159|
|         SO57066|
|         SO69506|
|         SO49123|
|         SO69467|
|         SO63195|
|         SO65308|
|         SO57175|
|         SO48764|
|         SO51789|
|         SO49859|
|         SO44101|
|         SO67311|
|         SO65310|
|         SO45568|
+----------------+
only showing top 20 rows



### 8. Identificar al empleado con mayor cantidad de ventas en cada mes del año 2020. id_empleado / nombre_empleado / mes / total_ventas

In [176]:
sales_employees = sales.join(salesperson, 'EmployeeKey')\
.select(
    'OrderDate',
    'EmployeeID',
    'Salesperson',
    'Sales'
)\
.filter(year('OrderDate') == 2020)\
.groupBy(
    col('EmployeeID').alias('id_empleado'),
    col('Salesperson').alias('nombre_empleado'),
    month('OrderDate').alias('mes')
).agg(sum('Sales').alias('total_ventas'))

In [177]:
SE_spec = Window.partitionBy('mes').orderBy(desc('total_ventas'))
sales_employees.withColumn('rn', row_number().over(SE_spec))\
.select(
    'id_empleado',
    'nombre_empleado',
    'mes',
    round('total_ventas',2).alias('total_ventas')
)\
.filter(col('rn') == 1).show(truncate=False)

PythonException: ignored

### 9. Mostrar las distintas categorias de productos que tiene la empresa

In [114]:
product.select('Category').distinct().show()

+-----------+
|   Category|
+-----------+
|      Bikes|
|   Clothing|
|Accessories|
| Components|
+-----------+



### 10. Identificar el total de unidades vendidas, total de ventas y el monto de la venta para cada categoria de producto. categoria / unidades_vendidas / total_ventas / monto_venta

In [178]:
(sales.join(product, 'ProductKey')\
.select(
    col('Sales').cast().alias('Sales'),
    'category',
    'SalesOrderNumber',
    'Quantity'
)\
.groupBy(col('category').alias('categoria')).\
agg(
    sum('Quantity').alias('unidades_vendidas'),
    count('SalesOrderNumber').alias('total_ventas'),
    sum('Sales').alias('monto_venta')
))\
.show()

AttributeError: ignored

In [179]:
category_spec = Window.partitionBy('category')

sales.join(product, 'ProductKey')\
.withColumn('unidades_vendidas', sum('Quantity').over(category_spec))\
.withColumn('total_ventas', count('SalesOrderNumber').over(category_spec))\
.withColumn('monto_venta', sum('Sales').over(category_spec))\
.show()

AttributeError: ignored